In [ ]:
from __future__ import print_function, division
from sklearn.datasets import fetch_20newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score,log_loss
from sklearn.metrics import roc_curve
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from nltk.stem.snowball import SnowballStemmer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import save,load,save_plot,plot_confusion_matrix

# Part (a) : Data Visualization

In [ ]:
categories = ['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'rec.autos',
              'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey']
train_data = {}
dic = {}
for i in range(len(categories)):
    temp_data = fetch_20newsgroups(subset='train', categories=categories[i:i+1], shuffle=True,
                                   random_state=42, remove=('headers','footers','quotes'))
    dic[i] = len(temp_data.data)
train_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True,
                                random_state=42, remove=('headers','footers','quotes'))
keys = []
vals = []

for key,val in dic.items():
    keys.append(key)
    vals.append(val)
    
fig = plt.figure(figsize=(15,15))
plotList = plt.bar(keys, vals, align='center', alpha=0.5)
color = ['r', 'g', 'b', 'grey', 'magenta', 'brown', 'pink', 'purple']
for i in range(len(keys)):
    plotList[i].set_color(color[i])
plt.xlabel('Categories',fontsize=15)
plt.xticks(keys, train_data.target_names, rotation=-45,fontsize=15)
plt.ylabel('Number of Records',fontsize=15)
plt.title('Histogram')
save_plot(fig,filename='histogram',directory='results')

# Part (b) : Calculating TF-IDF

In [ ]:
categories = ['comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','comp.sys.mac.hardware',
             'rec.autos','rec.motorcycles','rec.sport.baseball','rec.sport.hockey']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True,
                                  random_state=42, remove=('headers','footers','quotes'))
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True,
                                 random_state=42, remove=('headers','footers','quotes'))
print("Length of training data ==>", len(twenty_train.data))
print("Length of testing data ==>", len(twenty_test.data))

In [ ]:
def initParams(tfidf_min_df):
    stemmer = SnowballStemmer('english')
    analyzer = CountVectorizer().build_analyzer()
    #Use token_pattern parameter with analyzer='word' if no stemming and want to remove words that are only numbers.
    vectorizer = CountVectorizer(stop_words='english', min_df=tfidf_min_df, max_df=0.8, analyzer=stemmedWords,
                                 strip_accents='ascii', token_pattern='\w*[a-zA-Z]')
    tfidf_transformer = TfidfTransformer()
    return [stemmer,analyzer,vectorizer,tfidf_transformer]

def getTfidf(vectorizer,tfidf_transformer,data,isTraining=True):
    if(isTraining):
        count_data = vectorizer.fit_transform(data)
        tfidf_data = tfidf_transformer.fit_transform(count_data)
    else:
        count_data = vectorizer.transform(data)
        tfidf_data = tfidf_transformer.transform(count_data)        
    return tfidf_data

def stemmedWords(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

stemmer,analyzer,vectorizer,tfidf_transformer = initParams(tfidf_min_df=2)

load_from_previous = False

if(load_from_previous):
    training_data = load('tfidf_training')
    testing_data = load('tfidf_testing')
else:
    training_data = getTfidf(vectorizer,tfidf_transformer,twenty_train.data)
    testing_data = getTfidf(vectorizer,tfidf_transformer,twenty_test.data,isTraining=False)
    save(training_data,'tfidf_training')
    save(testing_data,'tfidf_testing')

print('Shape of Training Data ==>', training_data.shape)
print('Shape of Testing Data ==>', testing_data.shape)



In [ ]:
print(training_data.nnz / float(training_data.shape[0]))
min_df='2'

# Part (c) : Calculating TF-ICF

In [ ]:
train_all = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, remove=('headers','footers','quotes'))
categories = list(train_all.target_names)
category_train = {key:'\n '.join([train_all.data[i] for i in range(len(train_all.data)) 
                                  if train_all.target[i] == categories.index(key)]) for key in categories}
_,_,tfidf_vectorizer,tfidf_transformer = initParams(tfidf_min_df=1)
keys,values=[],[]
for k,v in category_train.items():
    keys.append(k)
    values.append(v)

tficf_data = getTfidf(tfidf_vectorizer,tfidf_transformer,values)
save(tficf_data,'tficf_data')


def topTfidfFeats(row, features, top_n=10):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

def topFeatsInDoc(Xtr, features, row_id, top_n=10):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return topTfidfFeats(row, features, top_n)

def plotTopTfidfFeatures(df):
    ''' Plot the data frames returned by the function plot_tfidf_classfeats(). '''
    fig = plt.figure(figsize=(14, 11), facecolor="w")
    x = np.arange(len(df))
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_frame_on(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.set_xlabel("Mean Tf-Idf Score", labelpad=16, fontsize=18)
    ax.set_title("label = " + str(df.label), fontsize=20)
    ax.ticklabel_format(axis='x', style='sci', scilimits=(-2,2))
    ax.barh(x, df.tfidf, align='center', color='#3F5D7D')
    ax.set_yticks(x)
    ax.set_ylim([-1, x[-1]+1])
    ax.set_xticks(np.arange(0,1,0.1))
    ax.set_xlim([0,1])
    yticks = ax.set_yticklabels(df.feature, fontsize=18)
    save_plot(fig,filename=df.label,directory='results/imp_words')
    
for i in range(len(keys)):
    df = topFeatsInDoc(tficf_data, tfidf_vectorizer.get_feature_names(), i, top_n=10)
    df.label = keys[i]
    plotTopTfidfFeatures(df)

# Part (d) : Feature Selection (Dimensionality Reduction)
Reduce the dimentionality of the corpus using the following two methods.
1. Latent Semantic Indexing (LSI)
2. Non-negative Matrix Factorization (NMF)
Compare the results for both methods.

In [ ]:
load_from_previous = False
#LSI
def getLSI(data,lsi=None,isTrain=True):
    if(isTrain):
        lsi = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
        lsi_data = lsi.fit_transform(data)
    else:
        lsi_data = lsi.transform(data)
    return lsi,lsi_data

if(load_from_previous):
    model = load('lsi_model_'+min_df)
    lsi = model['model']
    lsi_train = model['train']
else:    
    lsi,lsi_train = getLSI(training_data)
    _,lsi_test = getLSI(testing_data,lsi,isTrain=False)
    save({'model':lsi,'train':lsi_train,'test':lsi_test},'lsi_model_'+min_df)
print('Shape of LSI Training Data ==>', lsi_train.shape)

#NMF
def getNMF(data,nmf=None,isTrain=True):
    if(isTrain):
        nmf = NMF(n_components=50, init='random', random_state=42)
        nmf_data = nmf.fit_transform(data)
    else:
        nmf_data = nmf.transform(data)
    return nmf,nmf_data

if(load_from_previous):
    model = load('nmf_model_'+min_df)
    nmf = model['model']
    nmf_train = model['train']
else:    
    nmf,nmf_train = getNMF(training_data)
    _,nmf_test = getNMF(testing_data,nmf,isTrain=False)
    save({'model':nmf,'train':nmf_train,'test':nmf_test},'nmf_model_'+min_df)
print('Shape of NMF Training Data ==>', nmf_train.shape)

# Part (e) : Binary Classification using SVM

In [ ]:
load_from_previous = True
train_target = np.array((map(lambda x : 0 if x<4 else 1,twenty_train.target)))
target_names = ['Computer Technology','Recreational Activity']
test_target = np.array((map(lambda x : 0 if x<4 else 1,twenty_test.target)))
#twenty_test.target_names = ['Computer Technology','Recreational Activity']

if(load_from_previous):
    lsi_test = load('lsi_model_'+min_df)['test']
    nmf_test = load('nmf_model_'+min_df)['test']
else:
    _,lsi_test = getLSI(testing_data,lsi,isTrain=False)
    _,nmf_test = getNMF(testing_data,nmf,isTrain=False)

def runSVM(lsi_train,nmf_train,train_target,lsi_test,nmf_test,test_target,C=1000):
    global min_df
    fig=plt.figure()
    print('---------------LSI RESULTS---------------')
    clf = svm.SVC(kernel='linear',C=C,random_state=42,probability=True)
    clf.fit(lsi_train,train_target)
    y_pred_lsi = clf.predict(lsi_test)
    print('Accuracy of model =',accuracy_score(y_pred_lsi,test_target))
    print('Precision of model =',precision_score(y_pred_lsi,test_target))
    print('Recall of model =',recall_score(y_pred_lsi,test_target))
    print(classification_report(y_pred_lsi,test_target))
    cfm = confusion_matrix(y_pred_lsi,test_target)
    y_pred_lsi = clf.predict_proba(lsi_test)
    y_prob = [x[1] for x in y_pred_lsi]
    fpr, tpr, thresholds = roc_curve(test_target, y_prob, drop_intermediate=False)
    plt.plot(fpr,tpr)
    save_plot(fig,filename='lsi_svm_'+min_df+'_'+str(C),directory='results/roc')
    save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_lsi_svm_'+min_df+'_'+str(C),directory='results/confusion_matrix')

    fig=plt.figure()
    print('---------------NMF RESULTS---------------')
    clf = svm.SVC(kernel='linear',C=C,random_state=42,probability=True)
    clf.fit(nmf_train,train_target)
    y_pred_nmf = clf.predict(nmf_test)
    print('Accuracy of model =',accuracy_score(y_pred_nmf,test_target))
    print('Precision of model =',precision_score(y_pred_nmf,test_target))
    print('Recall of model =',recall_score(y_pred_nmf,test_target))
    print(classification_report(y_pred_nmf,test_target))
    cfm = confusion_matrix(y_pred_nmf,test_target)
    y_pred_nmf = clf.predict_proba(nmf_test)
    y_prob = [x[1] for x in y_pred_nmf]
    fpr, tpr, thresholds = roc_curve(test_target, y_prob,drop_intermediate=False)
    plt.plot(fpr,tpr)
    save_plot(fig,filename='nmf_svm_'+min_df+'_'+str(C),directory='results/roc')
    save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_nmf_svm_'+min_df+'_'+str(C),directory='results/confusion_matrix')

print('---------------HARD SVM---------------')
runSVM(lsi_train,nmf_train,train_target,lsi_test,nmf_test,test_target,C=1000)

print('---------------SOFT SVM---------------')
runSVM(lsi_train,nmf_train,train_target,lsi_test,nmf_test,test_target,C=0.001)

# Part (f) : K-Fold Cross Validation

In [ ]:
def kfold_cross_validation(train_data,train_target,test_data,test_target,typ):
    global min_df
    tuned_parameters = [{'kernel': ['linear'], 'C': [10**k for k in range(-3,4)]}]
    scores = ['accuracy']
    for score in scores:
        print("# Tuning hyper-parameters for {0} - {1}".format(score,typ))
        clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=5,
                           scoring=score)
        clf.fit(train_data, train_target)

        print("Best parameters set found on development set:")
        print(clf.best_params_)
        print("Grid scores on development set:")
        for params, mean_score, scores in clf.grid_scores_:
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean_score, scores.std() * 2, params))
        print("Detailed classification report:")
        y_pred = clf.predict(test_data)
        print('Accuracy of model =',accuracy_score(y_pred,test_target))
        print('Precision of model =',precision_score(y_pred,test_target))
        print('Recall of model =',recall_score(y_pred,test_target))
        print(classification_report(y_pred,test_target))
        cfm = confusion_matrix(y_pred,test_target)
        save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_best_kfold_'+typ+'_'+min_df,directory='results/confusion_matrix')

print('LSI')
kfold_cross_validation(lsi_train,train_target,lsi_test,test_target,'lsi')
print('NMF')
kfold_cross_validation(nmf_train,train_target,nmf_test,test_target,'nmf')


# Part (g) : Naive Bayes Classifier

In [ ]:
def plotROC(y_pred,test_target,filename):
    fig = plt.figure()
    fpr, tpr, thresholds = roc_curve(test_target, y_pred, drop_intermediate=False)
    plt.plot(fpr,tpr)
    save_plot(fig,filename,directory='results/roc')

print("------------------Gaussian Naive Bayes (LSI)------------------")
gnb = GaussianNB()
y_pred_lsi = gnb.fit(lsi_train, train_target).predict(lsi_test)
print('Accuracy of model =',accuracy_score(y_pred_lsi,test_target))
print('Precision of model =',precision_score(y_pred_lsi,test_target))
print('Recall of model =',recall_score(y_pred_lsi,test_target))
cfm = confusion_matrix(y_pred_lsi,test_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_gaussnb-lsi-'+min_df,directory='results/confusion_matrix')
plotROC(y_pred_lsi,test_target,'gaussnb-lsi-'+min_df)

print("------------------Gaussian Naive Bayes (NMF)------------------")
y_pred_nmf = gnb.fit(nmf_train, train_target).predict(nmf_test)
print('Accuracy of model =',accuracy_score(y_pred_nmf,test_target))
print('Precision of model =',precision_score(y_pred_nmf,test_target))
print('Recall of model =',recall_score(y_pred_nmf,test_target))
cfm = confusion_matrix(y_pred_nmf,test_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_gnb-nmf-'+min_df,directory='results/confusion_matrix')
plotROC(y_pred_nmf,test_target,'gnb-nmf-'+min_df)

print("------------------Multinomial Naive Bayes (NMF)------------------")
mnb = MultinomialNB()
y_pred = mnb.fit(nmf_train, train_target).predict(nmf_test)
print('Accuracy of model =',accuracy_score(y_pred_nmf,test_target))
print('Precision of model =',precision_score(y_pred_nmf,test_target))
print('Recall of model =',recall_score(y_pred_nmf,test_target))
cfm = confusion_matrix(y_pred_nmf,test_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_multinb-nmf-'+min_df,directory='results/confusion_matrix')
plotROC(y_pred_nmf,test_target,'multinb-'+min_df)

print("------------------Bernoulli Naive Bayes (NMF)------------------")
bnb = BernoulliNB()
y_pred = bnb.fit(nmf_train, train_target).predict(nmf_test)
print('Accuracy of model =',accuracy_score(y_pred_nmf,test_target))
print('Precision of model =',precision_score(y_pred_nmf,test_target))
print('Recall of model =',recall_score(y_pred_nmf,test_target))
cfm = confusion_matrix(y_pred_nmf,test_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_bernoullinb-'+min_df,directory='results/confusion_matrix')
plotROC(y_pred_nmf,test_target,'bernoullinb-'+min_df)

# Part (h) : Logistic Regression

In [ ]:
def logisticRegression(train,train_target,test,test_target,filename,penalty='l2',C=1.0):
    logres = LogisticRegression(C=C, penalty=penalty)
    y_pred = logres.fit(train,train_target).predict(test)
    print('Accuracy of model =',accuracy_score(y_pred,test_target))
    print('Precision of model =',precision_score(y_pred,test_target))
    print('Recall of model =',recall_score(y_pred,test_target))
    cfm = confusion_matrix(y_pred,test_target)
    save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_'+filename+'-'+penalty+str(C),directory='results/confusion_matrix')
    plotROC(y_pred,test_target,filename+'-'+penalty+str(C))

print("------------------Logistic Regression (LSI)------------------")
logisticRegression(lsi_train,train_target,lsi_test,test_target,'logres-lsi')

print("------------------Logistic Regression (NMF)------------------")
logisticRegression(nmf_train,train_target,nmf_test,test_target,'logres-nmf')

print("------------------Logistic Regression (LSI)------------------")
logisticRegression(lsi_train,train_target,lsi_test,test_target,'logres-lsi',C=.000000001)

print("------------------Logistic Regression (NMF)------------------")
logisticRegression(nmf_train,train_target,nmf_test,test_target,'logres-nmf',C=.000000001)


C = [10**k for k in range(-2,5)]
for c in C:
    print("------------------Logistic Regression (LSI C="+str(c)+")------------------")
    logisticRegression(lsi_train,train_target,lsi_test,test_target,'logres-lsi-'+str(c),C=c)


# Part (i) : Regularization for Logistic Regression
1. L2 regularization was done in Part (h).
2. In this part, we will do L1 regularization.

In [ ]:
print("------------------Logistic Regression (LSI)------------------")
logisticRegression(lsi_train,train_target,lsi_test,test_target,'logres-lsi-'+min_df,penalty='l1')

print("------------------Logistic Regression (NMF)------------------")
logisticRegression(nmf_train,train_target,nmf_test,test_target,'logres-nmf-'+min_df,penalty='l1')

print("------------------Logistic Regression (TFIDF)------------------")
logisticRegression(training_data,train_target,testing_data,test_target,'logres-tfidf',penalty='l1')

C = [10**k for k in range(-2,5)]
for c in C:
    print("------------------Logistic Regression (LSI C="+str(c)+")------------------")
    logisticRegression(lsi_train,train_target,lsi_test,test_target,'logres-lsi-'+min_df+str(c),C=c,penalty='l1')

# Part (j) : Multiclass Classification

In [ ]:
#Load data for multiclass and preprocess it.
categories = ['comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'misc.forsale','soc.religion.christian']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True,
                                  random_state=42, remove=('headers','footers','quotes'))
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True,
                                 random_state=42, remove=('headers','footers','quotes'))

stemmer,analyzer,vectorizer,tfidf_transformer = initParams(tfidf_min_df=2)
training_data = getTfidf(vectorizer,tfidf_transformer,twenty_train.data)
training_target = twenty_train.target
testing_data = getTfidf(vectorizer,tfidf_transformer,twenty_test.data,isTraining=False)
testing_target = twenty_test.target
nmf,nmf_train = getNMF(training_data)
_,nmf_test = getNMF(testing_data,nmf,isTrain=False)
lsi,lsi_train = getLSI(training_data)
_,lsi_test = getLSI(testing_data,lsi,isTrain=False)

In [ ]:
print(training_data.shape)

## Multiclass Naive Bayes 

In [ ]:
target_names = categories
print("------------------Bernoulli Naive Bayes------------------")
bnb = BernoulliNB()
y_pred = bnb.fit(nmf_train, training_target).predict(nmf_test)
print('Accuracy of model =',accuracy_score(y_pred,testing_target))
print('Precision of model =',precision_score(y_pred,testing_target, average=None))
print('Recall of model =',recall_score(y_pred,testing_target, average=None))
print(classification_report(y_pred,testing_target))
cfm = confusion_matrix(y_pred,testing_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_multiclass_bnb',directory='results/confusion_matrix')

print("------------------Gaussian Naive Bayes------------------")
bnb = GaussianNB()
y_pred = bnb.fit(nmf_train, training_target).predict(nmf_test)
print('Accuracy of model =',accuracy_score(y_pred,testing_target))
print('Precision of model =',precision_score(y_pred,testing_target, average=None))
print('Recall of model =',recall_score(y_pred,testing_target, average=None))
cfm = confusion_matrix(y_pred,testing_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_multiclass_gnb',directory='results/confusion_matrix')

print("------------------Multinomial Naive Bayes------------------")
bnb = MultinomialNB()
y_pred = bnb.fit(nmf_train, training_target).predict(nmf_test)
print('Accuracy of model =',accuracy_score(y_pred,testing_target))
print('Accuracy of model =',accuracy_score(y_pred,testing_target))
print('Precision of model =',precision_score(y_pred,testing_target, average=None))
print('Recall of model =',recall_score(y_pred,testing_target, average=None))
cfm = confusion_matrix(y_pred,testing_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_multiclass_mnb',directory='results/confusion_matrix')

## Multiclass SVM (One vs One)

In [ ]:
ovo_svc = OneVsOneClassifier(svm.SVC(kernel='linear',C=1000, random_state=42)).fit(lsi_train, training_target)
y_pred = ovo_svc.predict(lsi_test)
print('Accuracy of model =',accuracy_score(y_pred,testing_target))
print('Precision of model =',precision_score(y_pred,testing_target, average=None))
print('Recall of model =',recall_score(y_pred,testing_target, average=None))
cfm = confusion_matrix(y_pred,testing_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_multiclass_ovo_lsi',directory='results/confusion_matrix')

ovo_svc = OneVsOneClassifier(svm.SVC(kernel='linear',C=1000, random_state=42)).fit(nmf_train, training_target)
y_pred = ovo_svc.predict(nmf_test)
print('Accuracy of model =',accuracy_score(y_pred,testing_target))
print('Precision of model =',precision_score(y_pred,testing_target, average=None))
print('Recall of model =',recall_score(y_pred,testing_target, average=None))

cfm = confusion_matrix(y_pred,testing_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_multiclass_ovo_nmf',directory='results/confusion_matrix')

## Multiclass SVM (One vs Rest)

In [ ]:
ovr_svc = OneVsRestClassifier(svm.SVC(kernel='linear',C=1000, random_state=42)).fit(lsi_train, training_target)
y_pred = ovr_svc.predict(lsi_test)
print('Accuracy of model =',accuracy_score(y_pred,testing_target))
print('Precision of model =',precision_score(y_pred,testing_target, average=None))
print('Recall of model =',recall_score(y_pred,testing_target, average=None))
cfm = confusion_matrix(y_pred,testing_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_multiclass_ovr_lsi',directory='results/confusion_matrix')

ovr_svc = OneVsRestClassifier(svm.SVC(kernel='linear',C=1000, random_state=42)).fit(nmf_train, training_target)
y_pred = ovr_svc.predict(nmf_test)
print('Accuracy of model =',accuracy_score(y_pred,testing_target))
print('Precision of model =',precision_score(y_pred,testing_target, average=None))
print('Recall of model =',recall_score(y_pred,testing_target, average=None))
cfm = confusion_matrix(y_pred,testing_target)
save_plot(plot_confusion_matrix(cfm,target_names),
              filename='cfm_multiclass_ovr_nmf',directory='results/confusion_matrix')